In [1]:
!pip install theano
!pip install pymc3
!pip install sklearn
#!pip install matplotlib
#!pip install seaborn

import pandas as pd
import theano
from pymc3.theanof import set_tt_rng, MRG_RandomStreams
import pymc3 as pm
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np

In [2]:
floatX = theano.config.floatX

# read data and delete current loans

loan = pd.read_csv("data/loan.csv", low_memory=False, nrows=10000)
loan = loan[loan["loan_status"] != "Current"]

In [3]:
# select features and target

features = loan[["annual_inc", "int_rate", "purpose", "loan_amnt"]]
features = pd.get_dummies(features)
target = loan.apply(lambda x: ((x['total_pymnt'] - x['loan_amnt'])/x['loan_amnt']), axis = 1)
target = target.round(2)
target.head()
features.head()

,annual_inc,int_rate,loan_amnt,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,24000.0,10.65,5000.0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,30000.0,15.27,2500.0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,12252.0,15.96,2400.0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,49200.0,13.49,10000.0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,36000.0,7.90,5000.0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [4]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=42)

scaler_tr = preprocessing.MinMaxScaler().fit(x_train)
x_train = scaler_tr.transform(x_train)

scaler_test = preprocessing.MinMaxScaler().fit(x_test)
x_test = scaler_test.transform(x_test)

x_train = np.array(x_train.astype(floatX))
x_test = np.array(x_test.astype(floatX))
y_train = np.array(y_train.astype(floatX))
y_test = np.array(y_test.astype(floatX))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [12]:
def construct_nn_with_dropout(ann_input, ann_output, training):
    n_hidden = 100  # number of neurons per layer
    srng = MRG_RandomStreams(seed=12345)

    # Initialize random weights between each layer
    init_1 = np.random.randn(features.shape[1], n_hidden).astype(floatX)
    init_2 = np.random.randn(n_hidden, n_hidden).astype(floatX)
    init_3 = np.random.randn(n_hidden, n_hidden).astype(floatX)
    init_out = np.random.randn(n_hidden).astype(floatX)

    with pm.Model() as neural_network:
        # Weights from input to hidden layer
        weights_in_1 = pm.Normal('w_in_1', 0, sd=1,
                                 shape=(features.shape[1], n_hidden),
                                 testval=init_1)

        # Weights from 1st to 2nd layer
        weights_1_2 = pm.Normal('w_1_2', 0, sd=1,
                                shape=(n_hidden, n_hidden),
                                testval=init_2)

        # Weights from 2nd to 3rd layer
        weights_2_3 = pm.Normal('w_2_3', 0, sd=1,
                                shape=(n_hidden, n_hidden),
                                testval=init_3)

        # Weights from hidden layer to output
        weights_3_out = pm.Normal('w_3_out', 0, sd=1,
                                  shape=(n_hidden,),
                                  testval=init_out)

        # Build neural-network using tanh activation function
        act_1 = pm.math.tanh(pm.math.dot(ann_input,
                                         weights_in_1))
        act_2 = pm.math.tanh(pm.math.dot(act_1,
                                         weights_1_2))
        act_3 = pm.math.tanh(pm.math.dot(act_2,
                                         weights_2_3))
        act_out = pm.math.sigmoid(pm.math.dot(act_3,
                                              weights_3_out))
        if(training):
            act_1 = act_1 * srng.binomial(size=(n_hidden,),p=0.5)
            act_2 = act_2 * srng.binomial(size=(n_hidden,),p=0.5)
            act_3 = act_3 * srng.binomial(size=(n_hidden,),p=0.5)
            act_out = act_out * srng.binomial(size=(n_hidden,),p=0.5)
        else:
            act_1 = 0.5 * act_1
            act_2 = 0.5 * act_2
            act_3 = 0.5 * act_3
            act_out = 0.5 * act_out
            
        out = pm.Normal('out',
                        act_out,
                        observed=ann_output,
                        total_size=y_train.shape[0]  # IMPORTANT for minibatches
                        )
        return neural_network

In [13]:
# turn input and output into shared variables (this way, values can be changed later to switch to test data)
ann_input = theano.shared(x_train)
ann_output = theano.shared(y_train)
neural_network = construct_nn_with_dropout(ann_input, ann_output, True)
set_tt_rng(MRG_RandomStreams(42))

In [14]:
with neural_network:
    inference = pm.ADVI()
    approx = pm.fit(n=500, method=inference)
    
trace = approx.sample(draws=5000)

Average Loss = 19,287: 100%|██████████| 500/500 [00:44<00:00, 11.21it/s]
Finished [100%]: Average Loss = 19,269


In [15]:
# Replace arrays our NN references with the test data
ann_input.set_value(x_test)
ann_output.set_value(y_test)

with neural_network:
    ppc = pm.sample_posterior_predictive(trace, samples=200, progressbar=False)

# Use probability of > 0.5 to assume prediction of class 1
pred = ppc['out'].mean(axis=0)

In [16]:
import matplotlib.pyplot as plt
from sklearn import linear_model
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
!pip install scipy
from scipy.stats import kurtosis, skew


er = []
g = 0
for i in range(len(y_test)):
    #print( "actual=", y_test[i], " observed=", pred[i])
    x = (y_test[i] - pred[i]) **2
    er.append(x)
    g = g + x
    
x = 0
for i in range(len(er)):
   x = x + er[i]
print("er", sum(er))
print ("MSE", x / len(er))

v = np.var(er)
print ("variance", v)

print ("average of errors ", np.mean(er))

m = np.mean(y_test)
print ("average of observed values", m)

y = 0
for i in range(len(y_test)):
    y = y + ((y_test[i] - m) ** 2)
    

#print ("total sum of squares", y)
#print ("ẗotal sum of residuals ", g)
#print ("r2 calculated", 1 - (g / y))
print("kurtosis", kurtosis(er))
print("skewness", skew(er))



er 322.04810334884115
MSE 0.10788881184215784
variance 0.055376624286545724
average of errors  0.10788881184215787
average of observed values 0.07008375209380234
kurtosis 17.551297598929146
skewness 3.834567518503511
